In [ ]:
!pip install asyncio truecallerpy
!pip install blockcypher
!pip install gradio
!pip install bhimupipy

SMS SPAM DETECTION

In [17]:
import pandas as pd
import re
import nltk
import pickle
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# Download nltk resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Load the trained model and vectorizer
with open('spam_classifier.pkl', 'rb') as file:
    vectorizer, model = pickle.load(file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rohis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Python310\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stab

In [18]:
def sms_detection(new_messages):
    # Preprocessing function
    def preprocess_text(text):
        stemmer = PorterStemmer()
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = text.split()
        text = [stemmer.stem(word) for word in text if word not in set(stopwords.words('english'))]
        return ' '.join(text)

    new_messages_processed = [preprocess_text(msg) for msg in new_messages]
    X_new = vectorizer.transform(new_messages_processed).toarray()
    predictions_new = model.predict(X_new)

    print("Predictions on new messages:")
    for msg, pred in zip(new_messages, predictions_new):
        print(f"Message: {msg} -- Predicted Label: {'spam' if pred == 1 else 'ham'}")

        return 'spam' if pred==1 else 'genuine'


In [19]:
new_messages = ["Congratulations! You've won a free trip. Reply to claim your prize."]
print(sms_detection(new_messages))

Predictions on new messages:
Message: Congratulations! You've won a free trip. Reply to claim your prize. -- Predicted Label: spam
spam


URL SPAM DETECTION

In [20]:
def extractUrl(data):
        url = str(data)
        extractSlash = url.split('/')
        result = []

        for i in extractSlash:
            extractDash = str(i).split('-')
            dotExtract = []

            for j in range(0,len(extractDash)):
                extractDot = str(extractDash[j]).split('.')
                dotExtract += extractDot

            result += extractDash + dotExtract
        result = list(set(result))

        return result

In [21]:
def url_detection(new_urls):
    # Load the trained model
    with open('url_spam_classifier.pkl', 'rb') as file:
        cv_loaded, nbModel_loaded = pickle.load(file)

    # Transform new URLs into features using the loaded CountVectorizer
    features_new = cv_loaded.transform(new_urls)

    # Make predictions using the loaded Multinomial Naive Bayes classifier
    predictions_new = nbModel_loaded.predict(features_new)

    print("Predictions on new URLs:")
    for url, pred in zip(new_urls, predictions_new):
        print(f"URL: {url} -- Predicted Label: {'spam' if pred == 1 else 'safe'}")
        return 'spam' if pred == 1 else 'safe'


In [22]:
new_urls = ["http://www.some-url.com"]
print(url_detection(new_urls))

Predictions on new URLs:
URL: http://www.some-url.com -- Predicted Label: spam
spam


c:\Python310\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


CALL DETECTION

In [23]:
import asyncio
import threading
from truecallerpy import search_phonenumber

# Enter Your country code and Truecaller installation ID:
COUNTRY_CODE = "IN"
INSTALLATION_ID = "a1i09--lrIx0j-lFqMXeAfLdZozX1gZ4OQS7FptTEpMMI4orsdpA3kwIV7CamFo8"

# Global variable to store the result
phone_number_result = ""

# Fetching Data:
async def get_truecaller_info(phone_number):
    try:
        # Perform the phone number search
        response = await search_phonenumber(phone_number, COUNTRY_CODE, INSTALLATION_ID)

        # Extract and store specific information
        if response.get("status_code") == 200:
            data = response.get("data", {}).get("data", [])[0]
            if data:
                name = data.get("name")
                if "spam" in name.lower() or "telemarketing" in name.lower():
                    return "Spam Number"
                else:
                    return "Verified Number"
            else:
                return "No Truecaller information found for the provided phone number."
        elif response.get("status_code") == 429:  # Too Many Requests
            return "Too Many Requests: Please try again later."
        else:
            return "Failed to retrieve Truecaller information. Status code:", response.get("status_code")

    except Exception as e:
        return f"An error occurred: {e}"

def run_async_function(phone_number_input):
    global phone_number_result
    phone_number_result = asyncio.run(get_truecaller_info(phone_number_input))

def check_no(phno):
    # Create a new thread and run the asynchronous function inside it
    thread = threading.Thread(target=run_async_function, args=(phno,))
    thread.start()
    thread.join()  # Wait for the thread to complete
    return phone_number_result  # Return the result obtained from the global variable

# Example usage:
phone_number = "7812827300"  # Replace with the phone number you want to check
result = check_no(phone_number)
print("Returned value:", result)  # Print the returned value


Returned value: Verified Number


BITCOIN VERFICATION

In [24]:
import blockcypher

def is_address_available(addr):
    try:
        blockcypher.get_total_balance(addr)
        return True
    except:
        return False

def bitcoin(addr):
    if is_address_available(addr):
       return "Address is available"
    else:
        return "Address is not available"


In [25]:
addr = "1234567"
bitcoin(addr)

'Address is not available'

UPI VERIFICATION

In [28]:
def upi_verification(st):
    %run upi.py
    return "Valid" if upivalue else "Not Valid"

Predictions on new messages:
Message: hello -- Predicted Label: ham
Predictions on new URLs:
URL: https://www.google.com/ -- Predicted Label: spam


c:\Python310\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


None


UI

In [29]:
import gradio as gr

def check_output(i1, i2, i3, i4, i5):
    a = check_no(i1)
    b = sms_detection([i2])
    c = url_detection([i3])
    d = upi_verification(i4)
    e = bitcoin(i5)

    return a, b, c, d, e



i1 = gr.Textbox(lines=1, label="Mobile Number")
i2 = gr.Textbox(lines=1, label="SMS")
i3 = gr.Textbox(lines=1, label="URL Link")
i4 = gr.Textbox(lines=1, label="UPI")
i5 = gr.Textbox(lines=1, label="Bitcoin Wallet")

app = gr.Interface(fn=check_output, inputs=[i1, i2, i3, i4, i5], outputs=["text", "text", "text", "text", "text"], title="Citizen Safety App")
app.launch()


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Predictions on new messages:
Message: hello -- Predicted Label: ham
Predictions on new URLs:
URL: https://briefingday.com/fan -- Predicted Label: spam


c:\Python310\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


None
